In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,saw,saw
band,[11],9
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,1e-24
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,[0]
ng_refs,-,[5]
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.000266                    1 -0.000004
0.000750        2 -0.000255                    2 -0.000020
0.001052        3 -0.000230                    3  0.000013
0.001476        4 -0.000197                    4  0.000028
0.002070        5 -0.000155                    5  0.000045
0.002904        6 -0.000104                    6  0.000066
0.004074        7 -0.000041                    7  0.000090
0.005714        8  0.000035                    8  0.000120
0.008015        9  0.000127                    9  0.000155
0.011243       10  0.000236                   10  0.000197
0.015771       11  0.000366                   11  0.000247
0.022122       12  0.000519                   12  0.000307
0.031031       13  0.000697                   13  0.000378
0.043528       14  0.000878                   14  0.000451
0.061057       15  0.001032                   15  0.000514
0.085645       16  0.001173                   16  0.000575
0.120136       17  0.001323                   17  0.000642
0.168516       18  0.001482                   18  0.000716
0.236378       19  0.001652                   19  0.000797
0.331549       20  0.001835                   20  0.000886
0.465100       21  0.002015                   21  0.000975
0.652400       22  0.001826                   22  0.000908
0.915100       23  0.001248                   23  0.000671
1.283650       24  0.000743                   24  0.000453
1.800600       25  0.000412                   25  0.000299
2.525700       26  0.000205                   26  0.000191
3.542800       27  0.000081                   27  0.000115
4.969550       28  0.000023                   28  0.000069
6.970850       29  0.000007                   29  0.000045
9.778100       30  0.000002                   30  0.000029
13.715850      31 -0.000003                   31  0.000016
19.239350      32 -0.000007                   32  0.000006
26.987250      33 -0.000005                   33  0.000002
37.855300      34  0.000002                   34  0.000004
53.100050      35  0.000008                   35  0.000007
73.887500      36  0.000013                   36  0.000009
97.662500      37  0.000016                   37  0.000010
121.437500     38  0.000014                   38  0.000008
145.212500     39  0.000013                   39  0.000003
168.987500     40  0.000008                   40 -0.000007
192.762500     41 -0.000003                   41 -0.000023
216.537500     42 -0.000020                   42 -0.000045
240.312500     43 -0.000043                   43 -0.000072
264.087500     44 -0.000076                   44 -0.000108
287.862500     45 -0.000124                   45 -0.000155
311.637500     46 -0.000157                   46 -0.000189
335.412500     47 -0.000140                   47 -0.000182
359.187500     48 -0.000083                   48 -0.000139
382.962500     49 -0.000014                   49 -0.000109
406.737500     50  0.000093                   50 -0.000052
430.512500     51  0.000241                   51  0.000080
454.287500     52  0.000439                   52  0.000289
478.062500     53  0.000694                   53  0.000543
501.837500     54  0.001012                   54  0.000775
525.612500     55  0.001396                   55  0.000983
549.387500     56  0.001842                   56  0.001226
573.162500     57  0.002345                   57  0.001594
596.937500     58  0.002889                   58  0.002138
620.712500     59  0.003455                   59  0.002809
644.487500     60  0.004022                   60  0.003638
668.262500     61  0.004567                   61  0.004467
692.037500     62  0.005080                   62  0.005271
715.812500     63  0.005556                   63  0.006015
739.587500     64  0.006008                   64  0.006688
763.362500     65  0.006456                   65  0.007300
78

# Fluxes by Layer

CRD                            CLIRAD                \
                       flug          fldg     fnetg      flug          fldg   
pressure    level                                                             
0.000000    1     -1.357878  0.000000e+00 -1.357878 -1.356040  7.629466e-09   
0.000624    2     -1.357878  6.161801e-09 -1.357878 -1.356040  9.474052e-09   
0.000876    3     -1.357878  8.887152e-09 -1.357878 -1.356040  1.096718e-08   
0.001229    4     -1.357878  1.349677e-08 -1.357878 -1.356040  1.391275e-08   
0.001723    5     -1.357878  2.129585e-08 -1.357878 -1.356040  1.812638e-08   
0.002417    6     -1.357878  3.445707e-08 -1.357878 -1.356040  2.436387e-08   
0.003391    7     -1.357878  5.661684e-08 -1.357878 -1.356040  3.383313e-08   
0.004757    8     -1.357878  9.382443e-08 -1.357878 -1.356040  4.854808e-08   
0.006672    9     -1.357878  1.561333e-07 -1.357878 -1.356040  7.200147e-08   
0.009359    10    -1.357878  2.602051e-07 -1.357878 -1.356040  1.098778e-07   
0.013128    11    -1.357878  4.335007e-07 -1.357878 -1.356040  1.717220e-07   
0.018415    12    -1.357878  7.211569e-07 -1.357878 -1.356040  2.734621e-07   
0.025830    13    -1.357878  1.197030e-06 -1.357877 -1.356040  4.416432e-07   
0.036232    14    -1.357878  1.980859e-06 -1.357876 -1.356040  7.201964e-07   
0.050823    15    -1.357878  3.242288e-06 -1.357875 -1.356040  1.171924e-06   
0.071291    16    -1.357878  5.197564e-06 -1.357872 -1.356039  1.880364e-06   
0.100000    17    -1.357877  8.169870e-06 -1.357868 -1.356038  2.977129e-06   
0.140271    18    -1.357875  1.266502e-05 -1.357862 -1.356037  4.676011e-06   
0.196760    19    -1.357872  1.942425e-05 -1.357852 -1.356035  7.307008e-06   
0.275997    20    -1.357866  2.951759e-05 -1.357837 -1.356031  1.137741e-05   
0.387100    21    -1.357857  4.445746e-05 -1.357813 -1.356026  1.767610e-05   
0.543100    22    -1.357841  6.615691e-05 -1.357775 -1.356017  2.733677e-05   
0.761700    23    -1.357820  9.231771e-05 -1.357728 -1.356007  3.997298e-05   
1.068500    24    -1.357799  1.160982e-04 -1.357683 -1.355995  5.331272e-05   
1.498800    25    -1.357779  1.343731e-04 -1.357645 -1.355985  6.639026e-05   
2.102400    26    -1.357762  1.466572e-04 -1.357615 -1.355977  7.908684e-05   
2.949000    27    -1.357748  1.529627e-04 -1.357595 -1.355970  9.128139e-05   
4.136600    28    -1.357737  1.538913e-04 -1.357583 -1.355965  1.028826e-04   
5.802500    29    -1.357730  1.516113e-04 -1.357579 -1.355963  1.143999e-04   
8.139200    30    -1.357726  1.491152e-04 -1.357577 -1.355963  1.269743e-04   
11.417000   31    -1.357724  1.480483e-04 -1.357576 -1.355966  1.413966e-04   
16.014700   32    -1.357727  1.492916e-04 -1.357578 -1.355974  1.580947e-04   
22.464000   33    -1.357738  1.545420e-04 -1.357583 -1.355989  1.778839e-04   
31.510500   34    -1.357759  1.696833e-04 -1.357589 -1.356013  2.043569e-04   
44.200100   35    -1.357790  2.043202e-04 -1.357586 -1.356048  2.447321e-04   
62.000000   36    -1.357836  2.671836e-04 -1.357569 -1.356096  3.075999e-04   
85.775000   37    -1.357898  3.650528e-04 -1.357533 -1.356163  3.993260e-04   
109.550000  38    -1.357961  4.717843e-04 -1.357489 -1.356231  4.956561e-04   
133.325000  39    -1.358029  5.802641e-04 -1.357449 -1.356305  5.915721e-04   
157.100000  40    -1.358136  7.246989e-04 -1.357411 -1.356420  7.157138e-04   
180.875000  41    -1.358295  9.054795e-04 -1.357389 -1.356586  8.636767e-04   
204.650000  42    -1.358522  1.124544e-03 -1.357398 -1.356821  1.032231e-03   
228.425000  43    -1.358821  1.367721e-03 -1.357454 -1.357125  1.208008e-03   
252.200000  44    -1.359207  1.632667e-03 -1.357575 -1.357511  1.390162e-03   
275.975000  45    -1.359712  1.923355e-03 -1.357789 -1.358010  1.584747e-03   
299.750000  46    -1.360388  2.250558e-03 -1.358137 -1.358666  1.804836e-03   
323.525000  47    -1.361261  2.681899e-03 -1.358579 -1.359503  2.110389e-03   
347.300000  48    -1.362310  3.335431e-03 -1.358974 -1.360509  2.604922e-0

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')